# Importing Dependencies

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.optimizers import Adam

In [34]:
cancer_dataset = pd.read_csv('../Dataset/dataset_with_missing_values.csv')

In [35]:
cancer_dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [36]:
cancer_dataset = cancer_dataset.drop(columns='id', axis=1)

# Checking The Shape Of The Dataset

In [37]:
cancer_dataset.shape

(5121, 31)

In [38]:
cancer_dataset.isnull().sum()

diagnosis                   0
radius_mean                 2
texture_mean                4
perimeter_mean              3
area_mean                   4
smoothness_mean            12
compactness_mean            5
concavity_mean             16
concave points_mean        12
symmetry_mean               9
fractal_dimension_mean      7
radius_se                  19
texture_se                  4
perimeter_se                7
area_se                     4
smoothness_se              14
compactness_se             12
concavity_se                7
concave points_se          11
symmetry_se                 7
fractal_dimension_se       11
radius_worst                2
texture_worst               5
perimeter_worst             7
area_worst                  2
smoothness_worst            4
compactness_worst           9
concavity_worst             4
concave points_worst        8
symmetry_worst              5
fractal_dimension_worst     6
dtype: int64

# Setting Threshold as 70%

In [39]:
cancer_dataset = cancer_dataset.dropna(thresh=cancer_dataset.shape[1]-9, axis=0)

In [40]:
cancer_dataset.shape

(5112, 31)

In [10]:
cancer_dataset.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5112.000000,5110.000000,5111.000000,5111.000000,5105.000000,5108.000000,5100.000000,5104.000000,5107.000000,5106.000000,...,5111.000000,5109.000000,5110.000000,5112.000000,5110.000000,5109.000000,5111.000000,5110.000000,5111.000000,5109.000000
mean,14.208070,19.291579,91.988445,654.870554,0.096707,0.104426,0.089079,0.049110,0.181158,0.062790,...,16.270562,25.682664,107.273599,880.701995,0.132375,0.254426,0.272382,0.114642,0.290091,0.083963
std,4.270376,4.300208,24.601481,351.751841,0.022238,0.052789,0.080629,0.040825,0.027390,0.007094,...,4.831485,6.144758,33.592748,569.147901,0.022828,0.157300,0.208531,0.065705,0.061834,0.018055
min,6.981000,9.710000,10.340000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.011110,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.100000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057693,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.093620,0.061550,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.850000,21.807500,104.100000,782.700000,0.105300,0.130400,0.131900,0.074040,0.195700,0.066142,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,96.570000,39.280000,356.100000,2501.000000,0.968200,0.345400,0.955900,0.956230,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [11]:
cancer_dataset.shape

(5112, 31)

# Categorical Encoding

B --> 0

M --> 1

In [12]:
cancer_dataset.replace({'diagnosis': {'B':0, 'M':1}}, inplace=True)

In [13]:
cancer_dataset.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# value counts

In [14]:
cancer_dataset['diagnosis'].value_counts()

0    3206
1    1906
Name: diagnosis, dtype: int64

In [15]:
cancer_dataset.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [16]:
cancer_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5112 entries, 0 to 5120
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                5112 non-null   int64  
 1   radius_mean              5112 non-null   float64
 2   texture_mean             5110 non-null   float64
 3   perimeter_mean           5111 non-null   float64
 4   area_mean                5111 non-null   float64
 5   smoothness_mean          5105 non-null   float64
 6   compactness_mean         5108 non-null   float64
 7   concavity_mean           5100 non-null   float64
 8   concave points_mean      5104 non-null   float64
 9   symmetry_mean            5107 non-null   float64
 10  fractal_dimension_mean   5106 non-null   float64
 11  radius_se                5100 non-null   float64
 12  texture_se               5109 non-null   float64
 13  perimeter_se             5109 non-null   float64
 14  area_se                 

In [17]:
cancer_dataset.groupby('diagnosis').mean()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
diagnosis,,,,,,,,,,,,,,,,,,,,,
0,12.212896,17.915546,78.026168,462.596818,0.092473,0.080155,0.046142,0.026018,0.174178,0.062870,...,13.377270,23.51852,86.994353,558.659295,0.124956,0.182739,0.166350,0.074433,0.270221,0.079455
1,17.564066,21.608577,115.466459,978.455643,0.103832,0.145199,0.161152,0.088018,0.192911,0.062655,...,21.135724,29.31947,141.391701,1422.396065,0.144857,0.374894,0.450678,0.182234,0.323503,0.091537


# Correlation Matrix

In [18]:
correlation = cancer_dataset.corr()

In [19]:
correlation

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
diagnosis,1.000000,0.606007,0.415270,0.736021,0.709186,0.247029,0.595974,0.690007,0.734293,0.330718,...,0.776617,0.456605,0.783097,0.733924,0.421589,0.590842,0.659419,0.793508,0.416742,0.323656
radius_mean,0.606007,1.000000,0.275646,0.819953,0.816445,0.103375,0.426031,0.558599,0.645207,0.132122,...,0.802930,0.254948,0.801593,0.778590,0.097881,0.353394,0.450075,0.618730,0.143311,0.017890
texture_mean,0.415270,0.275646,1.000000,0.326958,0.321440,-0.009458,0.236926,0.300320,0.270254,0.070972,...,0.352970,0.912265,0.358154,0.343799,0.077061,0.278073,0.301025,0.295499,0.105037,0.118705
perimeter_mean,0.736021,0.819953,0.326958,1.000000,0.979135,0.145950,0.550645,0.700853,0.797505,0.179386,...,0.962393,0.300051,0.963462,0.935376,0.147701,0.451281,0.557897,0.764648,0.186405,0.049663
area_mean,0.709186,0.816445,0.321440,0.979135,1.000000,0.127424,0.498136,0.677475,0.779056,0.151763,...,0.962712,0.287887,0.959112,0.959191,0.123796,0.390507,0.512525,0.721970,0.143905,0.004036
smoothness_mean,0.247029,0.103375,-0.009458,0.145950,0.127424,1.000000,0.431715,0.351272,0.346853,0.351839,...,0.151221,0.022199,0.164394,0.146034,0.525409,0.313469,0.298278,0.333085,0.245662,0.333790
compactness_mean,0.595974,0.426031,0.236926,0.550645,0.498136,0.431715,1.000000,0.875696,0.786156,0.602648,...,0.534931,0.248026,0.589808,0.509208,0.565357,0.865530,0.815829,0.815188,0.510067,0.687347
concavity_mean,0.690007,0.558599,0.300320,0.700853,0.677475,0.351272,0.875696,1.000000,0.864191,0.498739,...,0.679791,0.297957,0.720945,0.667513,0.446937,0.749316,0.876052,0.853781,0.409045,0.511165
concave points_mean,0.734293,0.645207,0.270254,0.797505,0.779056,0.346853,0.786156,0.864191,1.000000,0.437751,...,0.785755,0.271043,0.810115,0.766342,0.434261,0.633568,0.713158,0.862847,0.357821,0.350683
symmetry_mean,0.330718,0.132122,0.070972,0.179386,0.151763,0.351839,0.602648,0.498739,0.437751,1.000000,...,0.185949,0.090140,0.219393,0.177513,0.426341,0.473408,0.433859,0.430798,0.699563,0.438941


In [20]:
data = cancer_dataset.values

In [21]:
data

array([[ 1.     , 17.99   , 10.38   , ...,  0.2654 ,  0.4601 ,  0.1189 ],
       [ 1.     , 20.57   , 17.77   , ...,  0.186  ,  0.275  ,  0.08902],
       [ 1.     , 19.69   , 21.25   , ...,  0.243  ,  0.3613 ,  0.08758],
       ...,
       [ 1.     , 16.6    , 28.08   , ...,  0.1418 ,  0.2218 ,  0.0782 ],
       [ 1.     , 20.6    , 29.33   , ...,  0.265  ,  0.4087 ,  0.124  ],
       [ 0.     ,  7.76   , 24.54   , ...,  0.     ,  0.2871 ,  0.07039]])

In [22]:
data.shape

(5112, 31)

In [23]:
data.shape[1]

31

In [24]:
ix = [i for i in range(data.shape[1]) if i != 0] 

In [25]:
x, y = data[:, ix], data[:, 0]

# Summarizing Total Missing Values

In [26]:
print('Missing: %d' % sum(np.isnan(x).flatten())) 

Missing: 123


In [27]:
# Using KNNImputer to Impute Missing Values

In [28]:
# define imputer 
imputer = KNNImputer(n_neighbors=6)

In [29]:
#fit on the dataset
imputer.fit(x)

KNNImputer(n_neighbors=6)

In [30]:
# transforming the dataset
xtrans = imputer.transform(x) 

# Summarizing Total Missing After Imputation

In [31]:

print('Missing: %d' % sum(np.isnan(xtrans).flatten()))

Missing: 0


In [32]:
xtrans.shape

(5112, 30)